# Interacting with a Condynsate project
In this tutorial, we will introduce how to import a pre-made Condynsate project, how to build a controller, how to run the pre-made Condynsate project, and how to post-process the simulation data. The example project in this notebook is a wheel on an axle. The goal is to build a controller that makes the wheel point in a desired direction by applying torque to the axle. We will do this by designing a PD controller.

## Building the simulation environment
When starting any Python project, the first thing to do is import dependencies. In this project, we will need the ``ae353_wheel`` module from the ``condynsate`` package.

In [1]:
# Import the wheel project. This module is used to simulate, render, and plot a wheel dynamic system
import ae353_wheel

To start the project, we first create an instance of the wheel simulator. The simulator handles simulating the dynamic system, rendering it in real-time, and creating live plots that show us data from the simulation. It also collects data generated during the simulation and returns it to the user after the simulation is complete

Running this command will:
1. Create an instance of the wheel simulator class
2. Open a visualization window in your default internet browser (3D render of the dynamic system)
3. Open an animation window (real time plotting of system data)

In [2]:
# Create an instance of the wheel simulator
sim = ae353_wheel.Wheel_sim(use_keyboard=True)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


## Creating the controller
Now that the simulator is created, we can make our own controller. In the scope of AE353, all controllers will have the same format as the one below. Specifically, all controllers will be a class that includes **exactly** three functions:
```python
def __init__(self)
def reset(self)
def run(self, **kwargs)
```
Descriptions for these functions is provided below.

In [3]:
class Controller():
    def __init__(self):
        """
        Initializes the controller. You should initialize any member variables that
        the controller will use to their starting values. This function is called
        when a new instance of the Controller class is created.
    
        Parameters
        ----------
        None.
    
        Returns
        -------
        None.
        """
        self.prev_time = 0.0
        self.prev_error = 0.0

    
    def reset(self):
        """
        Resets the controller to an initial state. You should reset any variables
        that track states, times, etc. in this function. This function will be 
        called just before the simulation starts running and any time the simulation
        is reset.
    
        Parameters
        ----------
        None.
    
        Returns
        -------
        None.
        """
        self.prev_time = 0.0
        self.prev_error = 0.0

    
    def run(self, **kwargs):
        """
        Runs the controller. Data is passed from the simulation environment to the
        controller via kwargs, the controller calculates the inputs to the system,
        and then returns them via a return statement
    
        Parameters
        ----------
        **kwargs : Dictionary
            A dictionary of data passed from the simulation environment to the 
            controller. Specific data is extracted from kwargs via the statement:
            kwargs["KEYWORD"], where "KEYWORD" is a key to the dictionary. The keys
            of kwargs for this project are as follows:

            angle : Float
                The current angle of the wheel in radians

            target_angle : Float
                The current target angle of the wheel in radians

            time : Float
                The current time in the simulation in seconds

            P : Float
                The current proportional gain value

            D : Float
                The current derivative gain value
            
    
        Returns
        -------
        inputs : List with length equal to the number of inputs
            This is a list of the inputs calculated by the controller. In this
            project, there is only one input, the torque to be applied to the 
            wheel axle. Even though there is only one input in this project,
            it still must be placed in a list of length one.
        
        """
        # Calculate the error
        error = kwargs["angle"] - kwargs["target_angle"]

        # Estimate the derivate of the error with respect to time
        d_time = kwargs["time"] - self.prev_time
        if d_time <= 0.:
            d_error = 0.0
        else:
            d_error = (error - self.prev_error) / d_time

        # Update the previous time values
        self.prev_time = kwargs["time"]
        self.prev_error = error
        
        # Set the control gains
        P = kwargs["P"]
        D = kwargs["D"]
    
        # Calculate the torque
        torque = 0.0

        # Format and return the torque
        inputs = [torque]
        return inputs

After defining our controller class, we need to make a new instance of this class that we can pass to the simulator. This is done in the same way that we created a new instance of the simulation class above:

In [4]:
# Create an instance of our Controller class
controller = Controller()

## Running the simulation
Now we are ready to run the simulation and collect data. To do this we call the ``sim.run()`` function. This function takes an instance of the ``Controller`` class as an argument and returns ``data``. The parameters and return values of ``sim.run()`` are shown below.

#### Parameters

> **controller** : *member of Controller class*  
> Your controller that will generated inputs to the system.
> 
> **max_time** : *Float or None, optional*  
> The total amount of time the simulation is allowed to run. If set to None, the simulation will run until "ESC" is pressed on the keyboard. If the keyboard is disabled (Mac users), and max_time is set to None, the simulation will automatically terminate after 10 seconds of simulated time. The default value is None.
> 
> **initial_angle** : *Float, optional*  
> The initial angle of the wheel in radians. The wheel is set to this angle when the simulation starts and when the simulation is reset. The default value is 0.0.
> 
> **initial_target_angle** : *Float, optional*  
> The initial target angle in radians. This is set when the simulation starts and when the simulation is reset. Must be within the range [-pi, pi]. The default value is pi.
> 
> **initial_P** : *Float, optional*  
> The initial value of the proportional gain. This is set when the simulation starts and when the simulation is reset. Must be within the range [0.0, 10.0]. The default value is 0.0.
> 
> **initial_D** : *Float, optional*  
> The initial value of the derivative gain. This is set when the simulation starts and when the simulation is reset. Must be within the range [0.0, 10.0]. The default value is 0.0.


#### Returns

> **data** : *Dictionary of Lists*  
> A dictionary containing all relevant data generated during the simulation. Specific data is extracted via the statement: data["KEYWORD"], where "KEYWORD" is a key to the dictionary. The keys of data for this project are as follows:
> 
> **data["angle"]** : *List of Floats*  
> A list of all wheel angles during the simulation.
> 
> **data["target_angle"]** : *List of Floats*  
> A list of all target wheel angles during the simulation.
> 
> **data["time"]** : *List of Floats*  
> A list of the time during the simulation.
> 
> **data["P"]** : *List of Floats*  
> A list of all proportional gains set during the simulation.
> 
> **data["D"]** : *List of Floats*  
> A list of all derivative gains set during the simulation.
> 
> **data["torque"]** : *List of Floats*  
> A list of all torques applied during the simulation.

If enabled you can use the keyboard to interact with the simulation:  
* press **ENTER** to start the simulation
* press **BACKSPACE** to reset the simulation
* press **SPACE** to pause the simulation
* press **ESC** to end the simulation

You may also manually set control gains and target angles:  
* press and hold **a** to decrease the target angle
* press and hold **d** to increase the target angle
* press and hold **f** to decrease the proportional gain
* press and hold **r** to increase the proportional gain
* press and hold **g** to decrease the derivative gain
* press and hold **t** to increase the derivative gain

In [5]:
# Run the simulation and collect the simulation data
data = sim.run(controller,
               max_time=None)

PRESS ENTER TO START SIMULATION.
PRESS ESC TO QUIT.
PRESS SPACE TO PAUSE/RESUME SIMULATION.
PRESS BACKSPACE TO RESET SIMULATION.
CONTINUING...
QUITTING...


## Post-processing the simulation data
After the simulation is run, we can plot the data using the Matplotlib package. To start, we import matplotlib.

In [6]:
# Import the pyplot module from the matplotlib package
import matplotlib.pyplot as plt

Now we can make whichever plots we see fit to make sense of the simulation data. Here we plot the wheel angle and target angle versus time as well as the applied torque versus time.

In [7]:
# Create a figure that has two plots in it
fig, axes = plt.subplots(2, 1, figsize=(7.2, 5.4), sharex=True)

In [8]:
# Create a plot of the wheel angle and target wheel angle versus time on the first plot
axes[0].plot(data['time'], data['angle'], label="Angle", c='k', lw=2.5)
axes[0].plot(data['time'], data['target_angle'], label="Target", c='r', lw=2.5)
axes[0].set_ylabel("Angles [rad]", fontsize=12)
axes[0].tick_params(axis='y', labelsize=12)
axes[0].legend(fontsize=12, shadow=True)
axes[0].set_title("Angle vs. Time", fontsize=14)
axes[0].grid()

In [9]:
# Create a plot of the torque versus time on the second plot
axes[1].plot(data['time'], data['torque'], label="Torque", c='b', lw=2.5)
axes[1].set_xlabel("Time [s]", fontsize=12)
axes[1].set_ylabel("Torque [Nm]", fontsize=12)
axes[1].tick_params(axis='both', labelsize=12)
axes[1].set_title("Torque vs. Time", fontsize=14)
axes[1].grid()

In [10]:
# Show the figure
fig.tight_layout(pad=2.0)
plt.show()

Termination command detected. Terminating keyboard listener. Goodbye
